[link text](https://)Synopsis Multi Class, Multi Label transfer learning training script using a sparsely labeled data set - labels values correspond to positive that concept present (1), positive that the concept is not present (0) and uncertain if the concept is present at all (-1)

We use Mobilenet V2 with no top as our feature extractor and are experimenting with various optimizer and losses to work with missing labels.

This notebook is part of the Synopsis project (see http://github.com/synopsis) and based heavily off of the Tensorflow custom data loading and transfer learning ipynb examples.

See these resources for info:

* https://stats.stackexchange.com/questions/207794/what-loss-function-for-multi-class-multi-label-classification-tasks-in-neural-n

* https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

* https://github.com/keras-team/keras/issues/10371

# GCS Setup


In [0]:
# Configure GCS
project_id = 'dev-sphere-240918'
bucket_name = 'synopsis_cinemanet'

from google.colab import auth
auth.authenticate_user()

!gcloud config set project {project_id}

# Environment Setup / Dependencies

Note we require TF 1.15 for some of our (current) training set up, which requires us to install some dependencies. 

In [0]:
!pip uninstall -y tensorflow tensorflow-estimator tensorboard

In [0]:
# Add NVIDIA package repositories
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo yes | dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo apt-get update
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!sudo yes | dpkg -i nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!sudo apt-get update

# Install NVIDIA driver
# !sudo apt-get install --no-install-recommends nvidia-driver-418
# !sudo apt-get -y installnvidia-driver-418
# Reboot. Check that GPUs are visible using the command: nvidia-smi

# Install development and runtime libraries (~4GB)
!sudo apt-get install -y --allow-change-held-packages --no-install-recommends \
    cuda-10-0 \
    libcudnn7=7.6.2.24-1+cuda10.0  \
    libcudnn7-dev=7.6.2.24-1+cuda10.0


# Install TensorRT. Requires that libcudnn7 is installed above.
!sudo apt-get install -y --allow-change-held-packages -y --no-install-recommends libnvinfer5=5.1.5-1+cuda10.0 \
    libnvinfer-dev=5.1.5-1+cuda10.0


In [0]:
!sudo rm /usr/local/cuda
!sudo ln -s /usr/local/cuda-10.0 /usr/local/cuda

In [0]:
!ls -al /usr/local/

In [0]:
# Our TF PB to CoreML converter
!pip install coremltools --no-dependencies
!pip install tfcoreml --no-dependencies


In [0]:
# set up dependencies:
!pip install tensorflow-gpu==1.14


# Data Set and Training Below:

In [0]:
# load tensorflow
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

print(tf.__version__)

AUTOTUNE = tf.data.experimental.AUTOTUNE

print("Eager execution: {}".format(tf.executing_eagerly()))

tf.test.gpu_device_name()


In [0]:

# some defs:
IMG_SIZE = 224
MODEL_TYPE = "MOBILENET" # could also be "NASNET"
DATA_ROOT = "/tmp/Synopsis_Model_All_Concepts/"


In [0]:
# copy our data set
!gsutil -m cp -r gs://{bucket_name}/Synopsis_Model_All_Concepts.zip /tmp/


In [0]:
# Unzip our dataset to /tmp/
!unzip -qq /tmp/Synopsis_Model_All_Concepts.zip -d /tmp/

In [0]:
all_label_files = [
                  "shot_lighting.csv",
                  "color_saturation.csv",
                  "color_key.csv",
                  "color_theory.csv",
                  "color_tones.csv",
                  "shot_angle.csv",
                  "shot_focus.csv",
                  "shot_framing.csv",
                  "shot_level.csv",
                  "shot_location.csv",
                  "shot_subject.csv",
                  "shot_timeofday.csv",
                  "shot_type.csv",
                  "texture.csv",
                   ]
# masked loss is an attempt at having 'unknowns' for some concepts where data isnt exhaustively labeled
# for example, an image thats a city exterior may contain concepts like sidewalk / street / store as well, but arent marked as such
# since wedont have a huge labelling force

# its unclear if this is a win. Its likely questionable but we dont have exhaustively labeled data just yet... 

labels_using_masked_loss = [
                  True,
                  False,
                  False,
                  False,
                  False,
                  False,
                  False,
                  False,
                  False,
                  False,
                  False,
                  False,
                  False,  
                  False,
                   ]

# TODO: Refactor this to be a dictionary of label csv, loss function, and augmentation functions
# assign agumentations for each data set. Not all are appropriate.
# ... 

In [0]:
#download our label file
for label in all_label_files:
  label_file = DATA_ROOT + label
  !gsutil -m cp -r gs://{bucket_name}/Synopsis_Model_All_Concepts/{label} {label_file}
  # verify our MD5 so we know we are on an expected label file:
  !md5sum {label_file}

DATA SET HELPER METHODS

DATA AUGMENTATION HELPERS

See https://www.wouterbulten.nl/blog/tech/data-augmentation-using-tensorflow-data-dataset/

To be implemented


In [0]:
def augment_rotate(x: tf.Tensor) -> tf.Tensor:
    """Rotation augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    # Rotate 0, 90, 180, 270 degrees
    return tf.image.rot90(x, tf.random_uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

In [0]:
def augment_flip_horizontal(x: tf.Tensor) -> tf.Tensor:
    """Flip augmentation

    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x)

    return x

In [0]:
def augment_color(x: tf.Tensor) -> tf.Tensor:
    """Color augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

In [0]:
def augment_zoom(x: tf.Tensor) -> tf.Tensor:
    """Zoom augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    # Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_ind=np.zeros(len(scales)), crop_size=(32, 32))
        # Return a random crop
        return crops[tf.random_uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]


    choice = tf.random_uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))

In [0]:
def augment_flip_vertical(x: tf.Tensor) -> tf.Tensor:
    """Flip augmentation

    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_up_down(x)

    return x

In [0]:
def get_dataset(file_path,  BATCH_SIZE, NUM_EPOCHS, COLUMN_NAMES, **kwargs,):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=BATCH_SIZE, 
      na_value="?",
      num_epochs=NUM_EPOCHS,
      column_names=COLUMN_NAMES,
      ignore_errors=True, 
      shuffle=True, #TEMPORARY
      **kwargs)
  return dataset

In [0]:
# we need to split out our data set to matching file paths and the labels as a sparse vector for each label,
# containing 1, 0, -1 values for label concept present, label concept not present, dont know if present 

# for example, a CSV row like 
# color_key_blue/1-5.jpg,1,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
# should return:
# * a tensor containing a path like data_root + /color_key_blue/1-5.jpg, 
# * a tensor of values [1,0,0,0, -1 etc]

# a function that returns a path, ordered dict of only the 

import tensorflow.python.util
def split_csv_to_path_and_labels(csv_row_Dict):
  # print("Calling split_csv_to_path_and_label")
 
  filepath = csv_row_Dict.pop('filepath')
  data_root_tensor = tf.constant(DATA_ROOT)
  
  filepath = tf.strings.join([data_root_tensor, filepath], separator='')
  
  #make a new tensor with the values of the LABEL_NAMES keys but packed into a 0, len(LABEL_NAMES) array
  labels = tf.stack(list(csv_row_Dict.values()), axis=1)

  return filepath, labels


In [0]:
def augment_image(image, augmentations):
  if augmentations != None:
    # do augmentations
    for agumentation in augmentations:
      image = agumentation(image)
  return image

In [0]:
 # some image loading and normalization code:
 def preprocess_image(image, augmentations):
  image = tf.image.decode_jpeg(image, channels=3, dct_method="INTEGER_ACCURATE")
  image = tf.cast(image, tf.float32)

  # augment on our full size image if we have any
  # image = augment_image(image, None)

  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE], method='bilinear', preserve_aspect_ratio=False)

  image /= 255.0  # normalize to [0,1] range
  
  return image

In [0]:
def load_and_preprocess_single_image_from_tensor(path,augmentations = None):
   image = tf.io.read_file(path)
   if image == None:
    print("Unable to load image at path:" + path )
   return preprocess_image(image, augmentations)
    
# this method uses function map
def load_and_preprocess_image_batch(batch_of_paths, batch_of_labels):
  batch_of_images = tf.map_fn(load_and_preprocess_single_image_from_tensor, batch_of_paths, dtype=tf.float32)
  return batch_of_images, batch_of_labels



TRAINING HELPER METHODS

In [0]:
# add a custom loss function to manage our multi label, multi class -1 (unknown) values
#https://github.com/keras-team/keras/issues/3893

from tensorflow.keras import backend as K

MASK_VALUE = -1

def build_masked_loss(loss_function, mask_value=MASK_VALUE):
    """Builds a loss function that masks based on targets

    Args:
        loss_function: The loss function to mask
        mask_value: The value to mask in the targets

    Returns:
        function: a loss function that acts like loss_function with masked inputs
    """

    def masked_loss_function(y_true, y_pred):
        mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())
        return loss_function(y_true * mask, y_pred * mask)

    return masked_loss_function

def masked_accuracy(y_true, y_pred):
    
    dtype = K.floatx()
    total = K.sum(K.cast(K.not_equal(y_true, MASK_VALUE), dtype) )
    correct = K.sum(K.cast(K.equal(y_true, K.round(y_pred)), dtype) )
    
    return correct / total

In [0]:
# alternatively:
# taken from https://www.dlology.com/blog/how-to-multi-task-learning-with-missing-labels-in-keras/
def masked_loss_function(y_true, y_pred):
    mask = K.cast(K.not_equal(y_true, MASK_VALUE), K.floatx())
    return K.binary_crossentropy(y_true * mask, y_pred * mask)

In [0]:
# alternatively:
# from https://stats.stackexchange.com/questions/207794/what-loss-function-for-multi-class-multi-label-classification-tasks-in-neural-n
def abs_KL_div(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), None)
    y_pred = K.clip(y_pred, K.epsilon(), None)
    return K.sum( K.abs( (y_true- y_pred) * (K.log(y_true / y_pred))), axis=-1)

In [0]:
# alternatively:
#  https://github.com/keras-team/keras/issues/11749#issuecomment-498709628 :
def custom_sparse_categorical_accuracy(y_true, y_pred):
    return K.cast(K.equal(K.max(y_true, axis=-1),
                          K.cast(K.argmax(y_pred, axis=-1), K.floatx())),K.floatx())

In [0]:
# set up our model, MNASNet or MobileNet
from keras.metrics import categorical_accuracy

def build_model(LABEL_NAMES, use_masked_loss = True):
  # base_model = tf.keras.applications.NASNetMobile(input_shape=(IMG_SIZE, IMG_SIZE, 3),
  #                                                include_top=False,
  #                                                weights='imagenet')
  base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                                include_top=False,
                                                weights='imagenet')

  base_model.summary()

  base_model.trainable = False

  # Let's take a look to see how many layers are in the base model
  print("Number of layers in the base model: ", len(base_model.layers))

  # Fine tune from this layer onwards
  #fine_tune_at = 600

  # Freeze all the layers before the `fine_tune_at` layer
  #for layer in base_model.layers[:fine_tune_at]:
  #  layer.trainable =  False

  #base_model.summary()
  num_labels = len(LABEL_NAMES)
  print("Number of Labels in model " + str(num_labels))

  output = tf.keras.layers.Dense(num_labels, activation = 'sigmoid', name="cinemanet_output")

  model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.GlobalAveragePooling2D(),
    output])

  print(model.input.op.name)
  print(model.input)

  print(model.output.op.name)
  print(model.output)

  loss = 'binary_crossentropy'
  metrics = ['accuracy','categorical_accuracy']
  if use_masked_loss:
    loss = build_masked_loss(K.binary_crossentropy)
    metrics = [masked_accuracy, 'accuracy','categorical_accuracy']

# compile our model
  optimizer = tf.train.AdamOptimizer()
  model.compile(optimizer=optimizer,
                # loss=build_masked_loss(K.binary_crossentropy),
                loss = loss,
                metrics = metrics)

                # loss=build_masked_loss(K.categorical_crossentropy),
                
                # TODO: Figure out:
                #loss= tf.compat.v1.nn.sigmoid_cross_entropy_with_logits(labels=labels_tensor, logits=label_batch[0]),                           
                # TODO: Figure out:
                #loss = tf.keras.losses.sparse_categorical_crossentropy,
                
                # https://github.com/keras-team/keras/issues/11749
                # see categorical_accuracy vs accuracy 

                
  #               metrics=["accuracy"]
                # metrics=["sparse_categorical_accuracy"])
  #               metrics=[custom_sparse_categorical_accuracy])

#flag to run on tpu 
  # tpu = False
  # if tpu:
  #   tpu_grpc_url = "grpc://"+os.environ["COLAB_TPU_ADDR"]
    
  #   #connect the TPU cluster using the address 
  #   tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
    
  #   #run the model on different clusters 
  #   strategy = keras_support.TPUDistributionStrategy(tpu_cluster_resolver)
    
  #   #convert the model to run on tpu 
  #   model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)

  return model



In [0]:
# image data for MNAS / Mobilenet needs to be -1, 1
def change_range(image,label):
  return 2.0 * image -1.0, label


EXPORT HELPER CODE

In [0]:
def keras_model_path(date, model_name_without_extension):
  now = date.strftime('%Y_%m_%d_%H_%M_%S')
  return "/tmp/" + now + "/" + model_name_without_extension + '.h5'

def keras_json_path(date, model_name_without_extension):
  now = date.strftime('%Y_%m_%d_%H_%M_%S')
  return "/tmp/" + now + "/" + model_name_without_extension + '.json'

def keras_weights_path(date, model_name_without_extension):
  now = date.strftime('%Y_%m_%d_%H_%M_%S')
  return "/tmp/" + now + "/" + model_name_without_extension + '-weights.h5'

def pb_model_path(date, model_name_without_extension):
  now = date.strftime('%Y_%m_%d_%H_%M_%S')
  return ("/tmp/" + now + "/",   model_name_without_extension + '.pb')

def pb_classifier_path(date, model_name_without_extension):
  now = date.strftime('%Y_%m_%d_%H_%M_%S')
  return "/tmp/" + now + "/" + model_name_without_extension + '-classifier.pb'

def pb_feature_extractor_path(date, model_name_without_extension):
  now = date.strftime('%Y_%m_%d_%H_%M_%S')
  return "/tmp/" + now + "/" + model_name_without_extension + '-feature-extractor.pb'

def coreml_model_path(date, model_name_without_extension):
  now = date.strftime('%Y_%m_%d_%H_%M_%S')
  return "/tmp/" + now + "/" + model_name_without_extension + '.mlmodel'

def coreml_classifier_path(date, model_name_without_extension):
  now = date.strftime('%Y_%m_%d_%H_%M_%S')
  return "/tmp/" + now + "/" + model_name_without_extension + '-classifier.mlmodel'

def coreml_feature_extractor_path(date, model_name_without_extension):
  now = date.strftime('%Y_%m_%d_%H_%M_%S')
  return "/tmp/" + now + "/" + model_name_without_extension + '-feature-extractor.mlmodel'

def labels_model_path(date, model_name_without_extension):
  now = date.strftime('%Y_%m_%d_%H_%M_%S')
  return "/tmp/" + now + "/" + model_name_without_extension + '_labels.txt'


In [0]:
def export_model_keras(model, model_name_without_extension, date):
  # save as a 'Saved Model'
  #tf.keras.experimental.export_saved_model(model, model_path+"_saved_model")

  # WHEN USING FREEZE GRAPH  IN TF 1.14 ON SAVED MODEL:
  # AssertionError: cinemanet_output/Identity is not in graph

  # Save as a Keras H5 (two methods)
  # model.save(model_path + '.h5')
  tf.keras.models.save_model(model, keras_model_path(date, model_name_without_extension) , overwrite=True, save_format="h5")

  # TF 1.1.4 / TF.KERAS CANNOT LOAD THE FINAL OUTPUT FROM THE h5 - This is Bug #28668
  # TF 2.0 CAN LOAD THE TF.KERAS h5 WITH THE cinemanet_output/Identity node
  # HOWEVER TF 2.0 CANNOT RUN FREEZE GRAPH OR FREEZE SESSION DUE TO LACK OF SESSION / GRAPH API.
  # TF SAVED MODEL HOWEVER "WORKS" BUT THE RESULTING PROTOCOL BUFFER IS SOME WEIRD "I DID NOT ASK FOR THIS"
  # TF SERVING PB VERSION WITH NO OUTPUTS WHOSE GRAPH LOOKS TOTALLY DIFFERENT THAN MY INPUT
  # SECONDLY THIS SAVED MODEL FILE DOES NOT CONTAIN cinemanet_output/Identity AND NOR 
  # DOES ANY OTHER EXTERNAL TOOL HAVE THE ABILITY TO CONVER THIS TO A "I ASKED FOR THIS" PB
  # ALSO TF 1.14 WHICHD DOES SUPPORT TF SAVED MODEL FORMAT CANNOT CONVERT THIS TO A VALID OUTPUT
  # IT ERRORS WITH 
  # SURPRISE
  # AssertionError: cinemanet_output/Identity is not in graph


  # Save as a Keras JSON / JSON Weights
  model_json = model.to_json()
  with open( keras_json_path(date, model_name_without_extension), "w") as json_file:
      json_file.write(model_json)
  model.save_weights(keras_weights_path(date, model_name_without_extension))


In [0]:
# define our freeze graph method
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


def export_model_pb(model, model_name_without_extension, date):
  tf.reset_default_graph()
  K.set_learning_phase(0)

  restored_model = tf.keras.models.load_model( keras_model_path(date, model_name_without_extension), compile=True)

  # print(restored_model.input.op.name)
  # print(restored_model.input)

  # print(restored_model.output.op.name)
  # print(restored_model.output)
  # restored_model.summary()

  frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in restored_model.outputs])
  #save our PB to temp
  model_location_and_name_tuple = pb_model_path(date, model_name_without_extension)
  tf.train.write_graph(frozen_graph, model_location_and_name_tuple[0], model_location_and_name_tuple[1], as_text=False)


In [0]:
def segment_model_pb(model_name_without_extension, date):
    # segment the above PB file to 2 graphs, feature extractor and classifier
  # Load the TF graph definition
  #tf_model_path = "/tmp/" + model_name_without_extension+".pb"
  tf_model_path_tuple = pb_model_path(date, model_name_without_extension)
  tf_model_path = tf_model_path_tuple[0] + tf_model_path_tuple[1]

  with open(tf_model_path, 'rb') as f:
      serialized = f.read()

  tf.reset_default_graph()
  original_gdef = tf.GraphDef()
  original_gdef.ParseFromString(serialized)

  from tensorflow.python.tools import strip_unused_lib
  from tensorflow.python.framework import dtypes
  from tensorflow.python.platform import gfile

  input_extractor_node_names = ['mobilenetv2_1.00_224_input']
  output_extractor_node_names = ['global_average_pooling2d/Mean']

  input_classifier_nodes_names = ['global_average_pooling2d/Mean']
  output_classifier_node_names = ['cinemanet_output/Sigmoid']

  extractor_gdef = strip_unused_lib.strip_unused(
          input_graph_def = original_gdef,
          input_node_names = input_extractor_node_names,
          output_node_names = output_extractor_node_names,
          placeholder_type_enum = dtypes.float32.as_datatype_enum)

  classifier_gdef = strip_unused_lib.strip_unused(
          input_graph_def = original_gdef,
          input_node_names = input_classifier_nodes_names,
          output_node_names = output_classifier_node_names,
          placeholder_type_enum = dtypes.float32.as_datatype_enum)


  # Save it to an output file
  extractor_graph_file  = pb_feature_extractor_path(date, model_name_without_extension)
  classifier_graph_file  = pb_classifier_path(date, model_name_without_extension) 

  with gfile.GFile(extractor_graph_file, "wb") as f:
      f.write(extractor_gdef.SerializeToString())

  with gfile.GFile(classifier_graph_file, "wb") as f:
      f.write(classifier_gdef.SerializeToString())    
  

In [0]:
def convert_segment_to_coreml(model_name_without_extension, label_names, date):
  #convert our PB to an ML Model
  import tfcoreml as tf_converter

  extractor_graph_file  = pb_feature_extractor_path(date, model_name_without_extension)
  classifier_graph_file  = pb_classifier_path(date, model_name_without_extension) 

  feature_extractor_name = coreml_feature_extractor_path(date, model_name_without_extension)
  classifier_name = coreml_classifier_path(date, model_name_without_extension) 

  # output our feature extractor
  # input images -> 1056 element vector:
  tf_converter.convert(tf_model_path = extractor_graph_file,
                      mlmodel_path = feature_extractor_name,
                      output_feature_names = ['global_average_pooling2d/Mean:0'],
                      input_name_shape_dict = {'mobilenetv2_1.00_224_input:0' : [1,IMG_SIZE,IMG_SIZE,3]},
                      image_input_names ='mobilenetv2_1.00_224_input:0',
                      red_bias=-1,
                      green_bias=-1,
                      blue_bias=-1,
                      image_scale=2.0/255.0,
                      is_bgr = False,
                      # class_labels = LABEL_NAMES
                      )
  
  #export classifier
  tf_converter.convert(tf_model_path = classifier_graph_file,
                      mlmodel_path = classifier_name,
                      output_feature_names = ['cinemanet_output/Sigmoid:0'],
                      input_name_shape_dict = {'global_average_pooling2d/Mean:0' : [1,1280]},
                      class_labels = label_names
                      )


In [0]:
def export_labels(model_name_without_extension, label_names, date):
  label_path = labels_model_path(date, model_name_without_extension)
  text_file = open(label_path, "w")
  text_file.write(", ".join(label_names))
  text_file.close()


In [0]:
import os

def export_model(model, model_name_without_extension, label_names, date):

  export_model_keras(model, model_name_without_extension, date)
  export_model_pb(model, model_name_without_extension, date)
  segment_model_pb(model_name_without_extension, date)
  convert_segment_to_coreml(model_name_without_extension, label_names, date)
  export_labels(model_name_without_extension, label_names, date)
 

In [0]:
def save_exports_to_project(date):

  now = date.strftime('%Y_%m_%d_%H_%M_%S')

  !gsutil -m cp -r /tmp/{now} gs://{bucket_name}/Synopsis_Models/

  # extractor_graph_file  = "/tmp/" + model_name_without_extension+"_feature.pb"
  # classifier_graph_file  = "/tmp/" +  model_name_without_extension+"_classifier.pb"

  # feature_extractor_name = model_name_without_extension + "_feature.mlmodel"
  # classifier_name = model_name_without_extension + "_classifier.mlmodel"

  # extractor_graph_file  = pb_feature_extractor_path(date, model_name_without_extension)
  # classifier_graph_file  = pb_classifier_path(date, model_name_without_extension) 

  # feature_extractor_name = coreml_feature_extractor_path(date, model_name_without_extension)
  # classifier_name = coreml_classifier_path(date, model_name_without_extension) 


  # # COPY OUR H5
  # !gsutil -m cp -r /tmp/{model_name_without_extension}.h5 gs://{bucket_name}/Synopsis_Models/{now}/{model_name_without_extension}.h5
  # # COPY OUR JSON AND WEIGHTS
  # !gsutil -m cp -r /tmp/{model_name}.json gs://{bucket_name}/Synopsis_Models/{now}/{model_name}.json
  # !gsutil -m cp -r /tmp/{model_name}-weights.h5 gs://{bucket_name}/Synopsis_Models/{now}/{model_name}-weights.h5
  # #copy our PBs
  # !gsutil -m cp -r /tmp/{model_name}.pb gs://{bucket_name}/Synopsis_Models/{now}/{model_name}.pb
  # !gsutil -m cp -r {extractor_graph_file} gs://{bucket_name}/Synopsis_Models/{now}/{extractor_graph_file}
  # !gsutil -m cp -r {classifier_graph_file} gs://{bucket_name}/Synopsis_Models/{now}/{classifier_graph_file}
  # #copy our ML Models
  # #!gsutil -m cp -r /tmp/{model_name}.mlmodel gs://{bucket_name}/Synopsis_Models/{model_name}.mlmodel
  # !gsutil -m cp -r /tmp/{feature_extractor_name} gs://{bucket_name}/Synopsis_Models/{now}/{feature_extractor_name}
  # !gsutil -m cp -r /tmp/{classifier_name} gs://{bucket_name}/Synopsis_Models/{now}/{classifier_name}

In [0]:
 !ls -alh /tmp

In [0]:
#Clean up out tmp
# !rm -rf /tmp/Cinemanet-2019-*


## ***OUR DATA SET LOADING AND TRAINING METHOD***

In [0]:
ALL_LABELS = []

from tensorflow.python.framework import ops

def train_on_csv_file(label_file, model_name, date, use_masked_loss = True, augmentations = None):
  # reset all the state of tensorflow, keras an ipynb because you'll lose your mind otherwise:
  #tf.reset_default_graph()
  ops.reset_default_graph()
  tf.keras.backend.set_learning_phase(1) # 0 testing, 1 training mode

# preview contents of CSV to verify things are sane
  import csv

  def lenopenreadlines(filename):
      with open(filename) as f:
          return len(f.readlines())

  def csvheaderrow(filename):
    with open(filename) as f:
      reader = csv.reader(f)
      return next(reader, None)

  # !head {label_file}

  NUM_IMAGES = ( lenopenreadlines(label_file) - 1) # remove header

  COLUMN_NAMES = csvheaderrow(label_file)
  
  LABEL_NAMES = COLUMN_NAMES[:]
  LABEL_NAMES.remove("filepath")

  ALL_LABELS.extend(LABEL_NAMES)

  # make our data set
  BATCH_SIZE = 256
  NUM_EPOCHS = 30
  FILE_PATH = ["filepath"]
  
  LABELS_TO_PRINT = ' '.join(LABEL_NAMES)
  print("Label contains: " + str(NUM_IMAGES) + " images")
  print("Label Are: " + LABELS_TO_PRINT)
  print("Creating Data Set From " + label_file)

  csv_dataset = get_dataset(label_file, BATCH_SIZE, NUM_EPOCHS, COLUMN_NAMES)

  #make a new data set from our csv by mapping every value to the above function
  split_dataset = csv_dataset.map(split_csv_to_path_and_labels)  

  # make a new datas set that loads our images from the first path 
  image_and_labels_ds = split_dataset.map(load_and_preprocess_image_batch, num_parallel_calls=AUTOTUNE)

  # update our image floating point range to match -1, 1
  ds = image_and_labels_ds.map(change_range)
  
  print(image_and_labels_ds)

  model = build_model(LABEL_NAMES, use_masked_loss)

  #split the final data set into train / validation splits to use for our model.
  DATASET_SIZE = NUM_IMAGES

  ds = ds.repeat()

  # train_size = int(0.8 * DATASET_SIZE)
  # val_size = int(0.1 * DATASET_SIZE)
  # test_size = int(0.1 * DATASET_SIZE)

  # print("train size: " + str(train_size) + ", validation size: " + str(val_size) + ", test_size: " + str(test_size))

  # train_dataset = ds.take(train_size)
  # test_dataset = ds.skip(train_size)
  # val_dataset = ds.skip(val_size)
  # test_dataset = ds.take(test_size)

  import math
  #quick - hack to speed up training just to test our model export pipeline...
  # steps_per_epoch = 100
  # val_steps_per_epoch = 100
  # epochs = 1
  # history = model.fit(ds, epochs=epochs, steps_per_epoch=steps_per_epoch)

  # steps_per_epoch =  int(math.floor(train_size/BATCH_SIZE))

  # seems like we have REALLY long validation times. Some folks report issues with this on Tensorflow / Keras git.
  # make it smaller just for now (extra / NUM_EPOCHS)
  # val_steps_per_epoch = int(math.floor(val_size/BATCH_SIZE) / NUM_EPOCHS)
  # val_steps_per_epoch = int(math.floor(val_size/BATCH_SIZE) / NUM_EPOCHS)

  # history = model.fit(train_dataset, epochs=NUM_EPOCHS, steps_per_epoch=steps_per_epoch, validation_data=val_dataset, validation_steps=val_steps_per_epoch, validation_freq=NUM_EPOCHS)

  steps_per_epoch =  int(math.floor(DATASET_SIZE/BATCH_SIZE))
  history = model.fit(ds, epochs=NUM_EPOCHS, steps_per_epoch=steps_per_epoch)


  print(history)

  # results = model.evaluate(test_dataset)
  # print('test loss, test acc:', results)
  export_model(model, model_name, LABEL_NAMES, date)

In [0]:
# save our model to temp
import datetime
from google.colab import output

date = datetime.datetime.now()
now = date.strftime('%Y_%m_%d_%H_%M_%S')
os.mkdir("/tmp/" + now )


#for label in all_label_files:
for i in range(len(all_label_files)):

  label = all_label_files[i]
  use_masked_loss = labels_using_masked_loss[i]

  label_destination = "/tmp/Synopsis_Model_All_Concepts/"
  label_file =  label_destination + label

  model_name ="CinemaNet_" + label  
  model_name = model_name.replace(".csv", "")

  print("!!!!!!!")
  print("!!!!!!!")
  print("!!!!!!!")

  print("Training Model: " + model_name)

  print("!!!!!!!")
  print("!!!!!!!")
  print("!!!!!!!")

  train_on_csv_file(label_file, model_name, date, use_masked_loss)
  
  output.clear()

In [0]:
# export to GCS once we are done with everything
save_exports_to_project(date)


In [0]:
# ensure every filepath has a file, otherwise we may get wierd errors like so:
# https://github.com/kratzert/finetune_alexnet_with_tensorflow/issues/30

# import os
# iterator = split_dataset.make_one_shot_iterator()
# while True:
#   try:
#      filepaths, labels = iterator.get_next()
#      list_of_filepaths = tf.unstack(filepaths)
#      for filepath in list_of_filepaths:
#         filepath_string = filepath.numpy().decode('UTF-8')
#         if not os.path.isfile(filepath_string):
#           print(filepath_string)

#   except tf.errors.OutOfRangeError:
#     break



In [0]:
# Verify images are ok
from PIL import Image

def is_jpg_body(filename):
    try:
        i=Image.open(filename)
        is_jpg = i.format =='JPEG'
        if not is_jpg:
          print("unexpected image format: " + i.format)
        i.close()
        return  is_jpg
    except IOError:
        print("bad image at: " + filename)
        return False

def is_jpg_header(filename):
    data = open(filename,'rb').read(11)
#    if data[:4] != '\xff\xd8\xff\xe0': 
#      print("Bad data at offset 4 for: " + filename)
#      return False
    if data[6:] != 'JFIF\0': 
      print("Bad data at offset 6 for: " + filename)
      return False
    return True
def check_tf_jpeg_decode(filename):
  try:
    image_contents = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_contents, channels=3)
  except:
    return False
  
  return True
 

In [0]:
# lets look at some of this

if tf.executing_eagerly():
  import matplotlib.pyplot as plt

  plt.figure(figsize=(8,8)) 
  image_batch, label_batch = next(iter(image_and_labels_ds))
  list_of_images = tf.unstack(image_batch, num=BATCH_SIZE)
  list_of_labels = tf.unstack(label_batch, num=BATCH_SIZE)


  print 
  for image in list_of_images:
  # #   plt.subplot(2,2,n+1)
    plt.imshow(image)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.show()



In [0]:
# predict on our model
import os
from urllib.parse import urlparse
#would be smarter to batch but whatever man
def predict_from_url(url, confidence):
  
  parsed_url = urlparse(url)

  image_file_name = os.path.basename(parsed_url.path)
  image = tf.keras.utils.get_file(origin=url, fname=image_file_name)

  image = load_and_preprocess_single_image_from_tensor(image)
  pred_image = image[np.newaxis,...] # dimension added to fit input size

  plt.imshow(image)
  plt.show()

  prediction = model.predict(pred_image)
  
  prediction_dict = dict(zip(LABEL_NAMES, prediction[0]))

  np.set_printoptions(precision=4)

  tags = ""
  for key in prediction_dict:
    value = prediction_dict[key]
    if value > confidence:
      tags = tags + key + ", "
    
    
  print(tags)

def predict_from_urls(urls, confidence):
  for u in urls:
    predict_from_url(u, confidence)

In [0]:

urls = ["http://uratex.com.ph/uratex-blog-2015/blog/wp-content/uploads/2015/02/sofa-bed-9.jpg",
        "https://i.kinja-img.com/gawker-media/image/upload/c_scale,f_auto,fl_progressive,q_80,w_800/kkgdekuiltgl8q1owmip.jpg",
        "https://cdn.newsday.com/polopoly_fs/1.19213543.1528999894!/httpImage/image.jpeg_gen/derivatives/landscape_768/image.jpeg",
        "https://media.tacdn.com/media/attractions-splice-spp-674x446/06/74/1a/88.jpg",
        "https://pmcvariety.files.wordpress.com/2013/12/her1.jpg?w=1000",
        "https://i.ytimg.com/vi/EKmT9D2VwTk/maxresdefault.jpg",
        "http://essentialhome.eu/inspirations/wp-content/uploads/2018/06/Its-popcorn-time-the-greatest-movie-scenes-around-UK-4.jpg",
        "https://www.theglobeandmail.com/resizer/pNJQ2snLgDH5dsZ8LCg0IWNuKI4=/2048x0/filters:quality(80)/arc-anglerfish-tgam-prod-tgam.s3.amazonaws.com/public/QX3GDBKSUZD7DJC3EBKSX4GVJQ",
       ]

if tf.executing_eagerly():
  predict_from_urls(urls, 0.25)
